In [133]:
import cv2
import numpy as np
import numpy.typing
from queue import Queue
import matplotlib.pyplot as plt
import random

MatLike = cv2.typing.MatLike

In [134]:
def cautaPrimulPixel(imagine: MatLike) -> tuple[int, int]:
    rows, cols = imagine.shape[0], imagine.shape[1]
    for i in range(rows):
        for j in range(cols):
            if imagine[i, j] != 0:
                return (i, j)
    return None

In [135]:
def cautaContor(imagine: MatLike) -> list[int]:
    directii: list[tuple[int, int]] = [(0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1), (1, 0), (1, 1)]
    indexDirectie: int = 0

    pozitiaStart = cautaPrimulPixel(imagine)
    if pozitiaStart is None:
        return []

    eticheta: int = imagine[pozitiaStart]
    contor = [pozitiaStart]
    cod: list[int] = []
    pozitiaCurenta = pozitiaStart
    while True:
        # Rotirea se face în sensul invers al acelor de ceasornic
        urmatoareaDirectie: int = (indexDirectie + 6) % 8
        gasit: bool = False
        for _ in range(len(directii)):
            urmatoareaPozitie: tuple[int, int] = (pozitiaCurenta[0] + directii[urmatoareaDirectie][0], pozitiaCurenta[1] + directii[urmatoareaDirectie][1])
            if imagine[urmatoareaPozitie] == eticheta:
                contor.append(urmatoareaPozitie)
                pozitiaCurenta = urmatoareaPozitie
                indexDirectie = urmatoareaDirectie
                cod.append(indexDirectie)
                gasit = True
                break
            urmatoareaDirectie = (urmatoareaDirectie + 1) % 8
        
        if not gasit or pozitiaCurenta == pozitiaStart:
            break

    return contor

In [136]:
def trasaturiGeometrice(label: int, labels: MatLike):
    arie: float = 0
    perimetru: float = 0
    numarVecini: int = 0
    height, width = labels.shape[0], labels.shape[1]

    for i in range(height):
        for j in range(width):
            if labels[i, j] == label:
                arie += 1
                if i - 1 >= 0 and labels[i-1, j] != 0: numarVecini += 1
                if i + 1 < height and labels[i+1, j] != 0: numarVecini += 1
                if j + 1 < width and labels[i, j+1] != 0: numarVecini += 1
                if j - 1 >= 0 and labels[i, j-1] != 0: numarVecini += 1
                perimetru += (4 - numarVecini)
                numarVecini = 0
    factorSubtiere: float = 4.0 * 3.14 * (arie / (perimetru**2))
    print("Aria: {}".format(arie))
    print("Perimetru: {}".format(perimetru))
    print("Factorul de subtiere: {}".format(factorSubtiere))

In [137]:
def mouseCallback(event, x, y, flags, param):
    image: MatLike = param['image']
    if event == cv2.EVENT_LBUTTONDOWN:
        label = image[y, x]
        if label != 0:
            trasaturiGeometrice(label, image)
        

In [138]:
import cv2
from cv2.typing import MatLike

def showImage(img: MatLike, name):
  cv2.imshow(name, img)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [139]:
def colorLabeledImage(image):
    if not isinstance(image, np.ndarray):
        raise ValueError("Input should be a numpy ndarray.")

    labels = np.unique(image)
    labels = labels[labels != 0]

    height, width = image.shape
    colored_image = np.zeros((height, width, 3), dtype=np.uint8)

    colors = {0: (255, 255, 255)}
    for label in labels:
        colors[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    for label, color in colors.items():
        colored_image[image == label] = color

    return colored_image
                

In [140]:
def showLabeledImageWithDifferentColors(labeledImage: MatLike, name: str):
  cv2.namedWindow(name)
  cv2.setMouseCallback(name, mouseCallback, param={'image': labeledImage})
  coloredImage = colorLabeledImage(labeledImage)
  cv2.imshow(name, coloredImage)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [141]:
import cv2
from cv2.typing import MatLike

def readImage(imagePath: str) -> MatLike:
    return cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)

In [142]:
def convertImageToBinary(image: MatLike) -> MatLike:
    grayScaleImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    showImage(grayScaleImage, 'GrayScale')

    _, binaryImage = cv2.threshold(grayScaleImage, 125, 255, cv2.THRESH_BINARY)
    showImage(binaryImage, 'Binary')

    return binaryImage


In [143]:
def labelBlackAndWhiteImage_traversaInLatime(image: MatLike) -> np.typing.NDArray:
    height, width = image.shape
    labels = np.zeros((height, width), dtype=int)
    label = 0

    for i in range(height):
        for j in range(width):
            if image[i, j] == 0 and labels[i, j] == 0:
                label += 1
                q = Queue()
                q.put((i, j))
                labels[i, j] = label

                while not q.empty():
                    x, y = q.get()
                    
                    neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
                    for nx, ny in neighbors:
                        if 0 <= nx < height and 0 <= ny < width:
                            if image[nx, ny] == 0 and labels[nx, ny] == 0:
                                q.put((nx, ny))
                                labels[nx, ny] = label

    return labels

In [144]:
def labelBlackAndWhiteImage_douaTreceriCuClaseDeEchivalenta(image: MatLike) -> np.typing.NDArray:
    height, width = image.shape
    labels = np.zeros((height, width), dtype=int)
    label = 0
    edges = {}

    neighbors = [(-1, 0), (0, -1), (-1, -1), (1, -1)]

    for i in range(height):
        for j in range(width):
            if image[i, j] == 0 and labels[i, j] == 0:
                l = set()
                for dx, dy in neighbors:
                    nx, ny = i + dx, j + dy
                    if 0 <= nx < height and 0 <= ny < width and labels[nx, ny] > 0:
                        l.add(labels[nx, ny])
                
                if not l:
                    label += 1
                    labels[i, j] = label
                else:
                    x = min(l)
                    labels[i, j] = x
                    for y in l:
                        if y != x:
                            edges.setdefault(x, set()).add(y)
                            edges.setdefault(y, set()).add(x)
    

    new_label = 0
    new_labels = np.zeros(label+1, dtype=int)
    
    for i in range(1, label):
        if new_labels[i] == 0:
            new_label += 1
            new_labels[i] = new_label
            Q = [i]
            while Q:
                x = Q.pop(0)
                for y in edges.get(x, []):
                    if new_labels[y] == 0:
                        new_labels[y] = new_label
                        Q.append(y)

    for i in range(height):
        for j in range(width):
            if labels[i, j] > 0:
                labels[i, j] = new_labels[labels[i, j]]

    return labels


In [145]:
import numpy
from skimage.filters import threshold_otsu

def binarize_image(grayscale_image):
    threshold_value = threshold_otsu(grayscale_image)
    binarized_image = grayscale_image > threshold_value
    binarized_image = (binarized_image * 255).astype(numpy.uint8)

    return binarized_image

In [146]:
def transformari_identitate(grayscale_image):
    return grayscale_image

In [147]:
def transformari_negative(grayscale_image):
    return 255 - grayscale_image

In [148]:
def transformari_contrast_modification(grayscale_image, in_min, in_max, out_min, out_max):
    image = numpy.clip(grayscale_image, in_min, in_max)
    return (image - in_min) * ((out_max - out_min) / (in_max - in_min)) + out_min

In [149]:
def transformari_gamma_correction(image, gamma):
    if gamma <= 0:
        raise ValueError("Gamma should be a positive number")
    return 255 * (image / 255) ** gamma

In [150]:
def transformari_brightness_adjustment(image, offset):
    return np.clip(image + offset, 0, 255)

In [151]:
import numpy

def histogram_equalization(image):
    hist, bins = numpy.histogram(image.flatten(), 256, [0,256])
    
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max() / cdf.max()
    
    cdf_m = numpy.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    cdf = numpy.ma.filled(cdf_m,0).astype('uint8')
    
    image_equalized = cdf[image]
    
    return image_equalized

In [152]:
def hysteresis(img):
    def sobel_edge_detection(image):
        # Aplică filtrul Sobel pentru detectarea muchiilor
        gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        magnitude = np.sqrt(gx**2 + gy**2)
        angle = np.arctan2(gy, gx) * 180 / np.pi
        return magnitude, angle

    def non_max_suppression(img):
        magnitude, angle = sobel_edge_detection(img)
        # Suprimarea non-maximelor
        M, N = magnitude.shape
        Z = np.zeros((M,N), dtype=np.float32)
        angle = angle % 180

        for i in range(1,M-1):
            for j in range(1,N-1):
                # Determină direcția muchiei
                try:
                    q = 255
                    r = 255

                    #angle 0
                    if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                        q = magnitude[i, j+1]
                        r = magnitude[i, j-1]
                    #angle 45
                    elif (22.5 <= angle[i,j] < 67.5):
                        q = magnitude[i+1, j-1]
                        r = magnitude[i-1, j+1]
                    #angle 90
                    elif (67.5 <= angle[i,j] < 112.5):
                        q = magnitude[i+1, j]
                        r = magnitude[i-1, j]
                    #angle 135
                    elif (112.5 <= angle[i,j] < 157.5):
                        q = magnitude[i-1, j-1]
                        r = magnitude[i+1, j+1]

                    if (magnitude[i,j] >= q) and (magnitude[i,j] >= r):
                        Z[i,j] = magnitude[i,j]
                    else:
                        Z[i,j] = 0

                except IndexError as e:
                    pass

        return Z

    def threshold(img, lowThresholdRatio=0.05, highThresholdRatio=0.09):
        highThreshold = img.max() * highThresholdRatio
        lowThreshold = highThreshold * lowThresholdRatio

        M, N = img.shape
        res = np.zeros((M,N), dtype=np.uint8)

        weak = np.int32(25)
        strong = np.int32(255)

        strong_i, strong_j = np.where(img >= highThreshold)
        zeros_i, zeros_j = np.where(img < lowThreshold)

        weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))

        res[strong_i, strong_j] = strong
        res[weak_i, weak_j] = weak

        return (res, weak, strong)

    img = non_max_suppression(img)
    img, weak, strong = threshold(img)
    M, N = img.shape
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img

In [153]:
def binarizare_adaptive(image, kernel_size = 3, C = 10):
    def sobel_edge_detection(image):
        # Filtru Sobel pentru detectarea muchiilor pe axele x și y
        sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        # Magnitudinea gradientului
        magnitude = np.sqrt(sobelx**2 + sobely**2)
        return magnitude

    image = sobel_edge_detection(image)
    rows, cols = image.shape
    binary_image = np.zeros((rows, cols), dtype=np.uint8)

    # Calculează pragul adaptiv și aplică binarizarea
    for i in range(rows):
        for j in range(cols):
            # Definirea ferestrei locale
            row_start = max(i - kernel_size // 2, 0)
            row_end = min(i + kernel_size // 2, rows)
            col_start = max(j - kernel_size // 2, 0)
            col_end = min(j + kernel_size // 2, cols)

            # Calcularea mediei locale
            local_mean = np.mean(image[row_start:row_end, col_start:col_end])
            # Binarizarea folosind pragul adaptiv
            if image[i, j] > local_mean - C:
                binary_image[i, j] = 255
            else:
                binary_image[i, j] = 0

    return binary_image

In [154]:
import numpy

def main():
    iuliaImagePath: str = "iulia.JPG"
    coinsImagePath: str = "coins.png"
    ticketsImagePath: str = "tickets.jpg"

    image: MatLike = readImage(ticketsImagePath)

    if image is None:
        print("Error reading image!")
        return 1

    showImage(image, "original")

    binarImage = hysteresis(image)
    showImage(binarImage, "Hysteresis")

if __name__ == "__main__":
    main()